In [1]:
import numpy as np
import pandas as pd

In [2]:
iot = pd.read_csv('Train_Test_Network.csv')
iot.head()

,ts,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,1554198358,3.122.49.24,1883,192.168.1.152,52976,tcp,-,80549.530260,1762852,41933215,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
1,1554198358,192.168.1.79,47260,192.168.1.255,15600,udp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
2,1554198359,192.168.1.152,1880,192.168.1.152,51782,tcp,-,0.000000,0,0,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
3,1554198359,192.168.1.152,34296,192.168.1.152,10502,tcp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
4,1554198362,192.168.1.152,46608,192.168.1.190,53,udp,dns,0.000549,0,298,...,0,0,-,-,-,bad_UDP_checksum,-,F,0,normal


In [3]:
iot.shape

(461043, 45)

In [73]:
iot['type'].unique()

array(['normal', 'scanning', 'dos', 'injection', 'ddos', 'password',
       'xss', 'ransomware', 'backdoor', 'mitm'], dtype=object)

In [ ]:
iot['weird_notice'].unique()

In [ ]:
iot['weird_addl'].unique()

In [ ]:
iot['http_resp_mime_types'].unique()

In [ ]:
iot.info()

In [ ]:
from sklearn import preprocessing
iot_ = iot.copy()
for i in range(5,len(iot_.columns)):
    if iot_.iloc[:,i].dtypes == 'object':
        le = preprocessing.LabelEncoder()
        le.fit(iot_.iloc[:,i])
        iot_.iloc[:,i] = le.transform(iot_.iloc[:,i])
iot_.info()

In [ ]:
iot_.head()

In [ ]:
len(iot_['dst_ip'].unique())

In [ ]:
srcip_list = ["172.16.0.1","172.17.0.1","172.18.0.1","172.19.0.1","172.20.0.1","172.21.0.1","172.22.0.1","172.23.0.1",
              "172.24.0.1","172.25.0.1","172.26.0.1","172.27.0.1","172.28.0.1","172.29.0.1","172.30.0.1","172.31.0.1"]
iot_['src_ip'] = np.random.choice(srcip_list, iot_.shape[0])

In [ ]:
iot_['src_ip'] = iot_['src_ip'].astype(str)+":"+iot_['src_port'].astype(str)
iot_['dst_ip'] = iot_['dst_ip'].astype(str)+":"+iot_['dst_port'].astype(str)
iot_.drop(columns=['src_port','dst_port'], inplace=True)
iot_.head()

In [ ]:
iot_.to_pickle("iot.pkl")

srcnode = set(iot_['src_ip'])
dtnode = set(iot_['dst_ip'])
nodes = list(dtnode.union(srcnode))
np.save("nodes.npy", nodes)                                                                                                                                                                                                                                                                                                                                                                                                      

binary = iot_['label'].to_numpy().astype(np.int8)
np.save("label_bi.npy", binary)

adj = iot_[['src_ip', 'dst_ip']].to_numpy()
np.save("adj.npy", adj)

multi = iot_['type'].to_numpy().astype(np.int16) 
np.save("label_mul.npy", multi)

edge_feat = iot_.iloc[:,list(range(5,43))].to_numpy()
np.save("edge_feat.npy",edge_feat)

In [ ]:
a = [0]
b = list(range(5,43))
c = a+b

In [ ]:
iot_['type'].unique()

In [ ]:
iot__ = iot_.copy()
iot__.iloc[:,c] = iot__.iloc[:,c].apply(lambda x: (x-x.mean())/ x.std(), axis=0)
iot__.head()

In [ ]:
edge_feat_scaled = iot__.iloc[:,c].to_numpy()
np.save("edge_feat_scaled.npy",edge_feat_scaled)

In [ ]:
import numpy as np
from sklearn.metrics import f1_score
label_bi = iot['label']
zeros = np.zeros((label_bi.shape))
f1_score(label_bi, zeros, average='micro')

In [ ]:
iot['label'].sum()/iot.shape[0]

In [ ]:
f1_score(label_bi, zeros, average='binary')

In [ ]:
f1_score(label_bi, zeros, average='weighted')

In [ ]:
f1_score(label_bi, zeros, average='macro')

In [ ]:
iot_['label'].unique()

In [ ]:
iot_['label'].sum()

## Edge adj

In [ ]:
import numpy as np
from collections import defaultdict
adj = np.load("adj.npy", allow_pickle=True)
adj_lists = defaultdict(set)  # not return keyerror when not found
for i, line in enumerate(adj):
    node1 = line[0]
    node2 = line[1]
    adj_lists[node1].add(i)  # mutual neighbor
    adj_lists[node2].add(i)
with open('adj_list.dict', 'wb') as file:
    pickle.dump(adj_lists, file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
import numpy as np
with open('adj_list.dict', 'rb') as file:
    adj_lists = pickle.load(file)
edge_idx = np.random.choice(np.arange(460000), 5)
adj = np.load("adj.npy", allow_pickle=True)
edge_idx.shape

In [5]:
import numpy as np
import pickle
from collections import defaultdict
adj = np.load("adj_random.npy", allow_pickle=True)
adj_lists = defaultdict(set)  # not return keyerror when not found
for i, line in enumerate(adj):
    node1 = line[0]
    node2 = line[1]
    adj_lists[node1].add(i)  # mutual neighbor
    adj_lists[node2].add(i)
with open('adj_random_list.dict', 'wb') as file:
    pickle.dump(adj_lists, file, protocol=pickle.HIGHEST_PROTOCOL)

In [61]:
import pickle
import numpy as np
with open('adj_random_list.dict', 'rb') as file:
    adj_lists = pickle.load(file)
edge_idx = np.random.choice(np.arange(460000), 2000)
adj = np.load("adj_random.npy", allow_pickle=True)
edge_idx.shape
# len(adj_lists)

(2000,)

In [5]:
def build_edge_index(source_nodes_ids, target_nodes_ids, seen_edges, edges_neigh):
    new_neigh = set()
    for edge in edges_neigh:
        nodes = adj[edge]
        for node in nodes:
            neigh = adj_lists.get(node)
            new_neigh = new_neigh.union(neigh)
            for edge_neigh in neigh:
                if (edge, edge_neigh) not in seen_edges and (edge_neigh, edge) not in seen_edges and edge != edge_neigh:
                    source_nodes_ids.append(edge)
                    target_nodes_ids.append(edge_neigh)
                    seen_edges.add((edge, edge_neigh))
                    
    return source_nodes_ids, target_nodes_ids, seen_edges, new_neigh

In [1]:
import numpy as np
adj = np.array([[0,1],[0,2],[1,2],[0,3],[1,3],[3,4],[2,5]])
adj_lists = {0:set([0,1,3]),1:set([0,2,4]),2:set([1,2,6]),
             3:set([3,4,5]),4:set([5]),5:set([6])}
edge_idx = np.arange(1)

In [6]:
import time
source_nodes_ids, target_nodes_ids = [], []
seen_edges = set()
edges = set(edge_idx)
edges_neigh = set(edge_idx)

a = time.time()
for i in range(2):
    source_nodes_ids, target_nodes_ids, seen_edges, edges_neigh = build_edge_index(source_nodes_ids, 
                                                                                   target_nodes_ids, seen_edges, edges_neigh)
    edges = edges.union(edges_neigh)
    print('finish')
b = time.time()

# source_nodes_ids.extend(edges)
# target_nodes_ids.extend(edges)

unique_map = {}
for idx, edge in enumerate(edges):
    unique_map[edge] = idx
    
source_nodes_ids = [unique_map[ids] for ids in source_nodes_ids]
target_nodes_ids = [unique_map[ids] for ids in target_nodes_ids]

# shape = (2, E), where E is the number of edges in the graph
edge_index = np.row_stack((source_nodes_ids, target_nodes_ids))
print(edge_index.shape, b-a)

finish
finish
(2, 12) 0.0


In [60]:
len(edges)

49189

In [7]:
edge_index

array([[0, 0, 0, 0, 1, 1, 1, 2, 2, 3, 3, 4],
       [1, 3, 2, 4, 3, 2, 6, 4, 6, 4, 5, 5]])

In [53]:
edge_idx

array([440459])

In [54]:
adj[440459]

array([ 50357, 149786])

In [55]:
adj_lists.get(50357)

{311209, 357029, 440459}

In [56]:
adj_lists.get(149786)

{224211, 262228, 390267, 422105, 440459}

In [4]:
print(edges)

{0, 1, 2, 3, 4, 5, 6}


In [7]:
edge_index

array([[0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4],
       [0, 1, 3, 2, 4, 1, 3, 2, 6, 2, 4, 6, 3, 4, 5, 4, 5]])

In [ ]:
len(adj_lists)

## adj_random

In [ ]:
edge_feat = np.load("edge_feat_scaled.npy")
srcip_list = list(range(100000))
dstip_list = list(range(100001,200000))
src = np.random.choice(srcip_list, edge_feat.shape[0])
dst = np.random.choice(dstip_list, edge_feat.shape[0])
src.shape

In [ ]:
adj_random = np.column_stack((src,dst))

In [ ]:
adj_random.shape

In [ ]:
np.save("adj_random.npy", adj_random)

In [ ]:
np.save("node_random.npy", np.array(list(range(200000))))

In [64]:
import torch
a = [1,2]
b = torch.tensor(a)
len(b)

2

In [65]:
import enum

In [86]:
label = np.load('label_mul.npy')
(label == 4).sum()/label.shape[0]

0.0022622618714523376

In [9]:
import torch

inpu = torch.randn(2, 3)
inpu

tensor([[-2.1830, -0.8871, -1.5883],
        [ 0.0115,  1.1728,  0.0585]])

In [21]:
3*(inpu - inpu.max())/(inpu.max()-inpu.min())

tensor([[-3.0000, -1.8414, -2.4683],
        [-1.0382,  0.0000, -0.9961]])

In [20]:
torch.topk(inpu,2,0)

torch.return_types.topk(
values=tensor([[ 0.0115,  1.1728,  0.0585],
        [-2.1830, -0.8871, -1.5883]]),
indices=tensor([[1, 1, 1],
        [0, 0, 0]]))

In [13]:
torch.tensor(70.0).exp()

tensor(2.5154e+30)

## Undersampling

In [8]:
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.model_selection import train_test_split
import numpy as np
from collections import Counter

In [35]:
edge_feat = np.load("edge_feat_scaled.npy")
label_bi = np.load("label_bi.npy")

In [36]:
num_edges = len(edge_feat)
train_val, test = train_test_split(np.arange(num_edges), test_size=140000, stratify=label_bi)
train_before, val = train_test_split(train_val, test_size=5000, stratify=label_bi[train_val])

In [37]:
print("Before undersampling: {}".format(Counter(label_bi[train_before])))
rus = RandomUnderSampler(random_state=42, sampling_strategy='auto')
train, y_rus = rus.fit_resample(train_before.reshape(-1, 1), label_bi[train_before])
print("After undersampling: {}".format(Counter(y_rus)))
train = train.reshape(-1,)

Before undersampling: Counter({0: 205649, 1: 110394})
After undersampling: Counter({0: 110394, 1: 110394})


In [38]:
edge_rus = np.concatenate((edge_feat[test], edge_feat[val], edge_feat[train]),axis=0)
edge_rus.shape

(365788, 39)

In [39]:
np.save('edge_feat_rus.npy', edge_rus)

In [40]:
label_bi_rus = np.concatenate((label_bi[test], label_bi[val], label_bi[train]),axis=0)
label_mul = np.load('label_mul.npy')
label_mul_rus = np.concatenate((label_mul[test], label_mul[val], label_mul[train]),axis=0)

In [41]:
np.save('label_bi_rus.npy', label_bi_rus)
np.save('label_mul_rus.npy', label_mul_rus)

In [42]:
srcip_list = list(range(90000))
dstip_list = list(range(90001,180000))
src = np.random.choice(srcip_list, edge_rus.shape[0])
dst = np.random.choice(dstip_list, edge_rus.shape[0])
adj_rus = np.column_stack((src,dst))
np.save('adj_rus.npy', adj_rus)

In [43]:
import pickle
from collections import defaultdict
adj = np.load("adj_rus.npy", allow_pickle=True)
adj_lists = defaultdict(set)  # not return keyerror when not found
for i, line in enumerate(adj):
    node1 = line[0]
    node2 = line[1]
    adj_lists[node1].add(i)  # mutual neighbor
    adj_lists[node2].add(i)
with open('adj_rus_list.dict', 'wb') as file:
    pickle.dump(adj_lists, file, protocol=pickle.HIGHEST_PROTOCOL)

In [44]:
Counter(label_mul)

Counter({5: 300000,
         8: 20000,
         2: 20000,
         3: 20000,
         1: 20000,
         6: 20000,
         9: 20000,
         7: 20000,
         0: 20000,
         4: 1043})